<a href="https://colab.research.google.com/github/apniinvention/projects/blob/main/Sentiment_Analysis_on_MongoDB_Document.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Objective** : To showcase how the Followers Count is Prortional to the sentiment of a user. Generally the user with high follower doesnt post negtive tweets.

1. Fetch all data from MongoDB
2. Iterate each data point & run the sentiment analysis
3. Once the sentiment prediction on tweet is done, update the document using its id
4. Go to MongoDB dashboard & draw the graph

In [1]:

# import the dependency
import re
import pymongo # for MongoDB
from textblob import TextBlob # for sentimental analysis 



In [2]:
# MongoDB Connection
url = "mongodb+srv://apniinvention:<password>@cluster0.qngzsud.mongodb.net/"
client = pymongo.MongoClient(url)

In [3]:
# Sentiment Analysis Code

# As the tweet might contain special character or some irrelevant data, so we need to clean before predicting the sentiment
def clean_text(text):
    text = str(text).lower()
    text = re.sub(r'#(\w+)', r'<\1>', text)
    text = re.sub(r'\n', ' ', text) # Remove /n
    text = re.sub(r'@[A-Za-z0-9]+', '<user>', text) # Remove and replace @mention
    text = re.sub(r'RT\s+', '', text) # Remove RT
    text = re.sub(r'https?://\S+\b/?', '<url>', text) # Remove and replace links
    return text

# sentiment prediction for each tweet
def senti_pred(text):
    # create TextBlob object of passed tweet text
    analysis = TextBlob(clean_text(text))
    # set sentiment
    if analysis.sentiment.polarity > 0:
      return 'positive'
    elif analysis.sentiment.polarity == 0:
      return 'neutral'
    else:
      return 'negative'


In [4]:
#update in mongoDB
def update_senti(id,sentiment):
  res = client.update_one({"_id":id},{"$set":{"sentiment":sentiment}},upsert=True)

  

In [78]:
#Iterate each document and run the sentiment prediction 

#connect to twitter database then tweets collection in mongodb
client = client["twitter"]["tweets"]

#get all data 
get_all = client.find({})
for each in get_all:
  
  if 'text' in each:
    
    sentiment = senti_pred(each['text']) # sentiment prediction
    update_senti(each['_id'],sentiment)  # update the existing mongodb document
    

  

Get Sentiment count


In [93]:
result = client.aggregate([
    {"$group" : {"_id":"$sentiment", "count":{"$sum":1}}}
])

In [94]:
# data under consideration
senti_result = {}
for each in result:
  senti_result[each['_id']] = each['count']
print(senti_result)

 
    

{'negative': 6310, 'positive': 17092, None: 11717, 'neutral': 16583}
